In [44]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [2]:
train_data = pd.read_feather("train_data.ftr")

In [45]:
train_data_2 = train_data

In [46]:
train_data_3 = train_data_2.sort_values(['customer_ID', 'S_2'], ascending = [True, False])
train_data_3.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
12,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-03-13,0.934570,0.009117,0.009384,1.007812,0.006104,0.135010,0.001604,0.007175,...,NaN,NaN,NaN,0.007187,0.004234,0.005085,NaN,0.005810,0.002970,0.008530
11,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-02-21,0.937500,0.002424,0.019836,1.008789,0.000607,0.102966,0.006172,0.007454,...,NaN,NaN,NaN,0.001563,0.001976,0.002569,NaN,0.001099,0.006348,0.004204
10,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-01-11,0.909668,0.005714,0.002829,1.004883,0.008179,0.098877,0.001853,0.003239,...,NaN,NaN,NaN,0.000427,0.004593,0.003613,NaN,0.007568,0.003004,0.006363
9,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-12-04,0.868652,0.001082,0.001930,1.007812,0.001772,0.100464,0.004627,0.006073,...,NaN,NaN,NaN,0.003611,0.009605,0.007267,NaN,0.008766,0.004753,0.009071
8,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-11-20,0.950684,0.009895,0.016891,1.003906,0.001789,0.102783,0.002520,0.009819,...,NaN,NaN,NaN,0.009743,0.003967,0.007942,NaN,0.008720,0.004368,0.000996


In [6]:
train_data_4 = train_data_3.drop_duplicates(subset=["customer_ID"], keep="first")
train_data_4.head()
# train_data_4.shape

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
12,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-03-13,0.934570,0.009117,0.009384,1.007812,0.006104,0.135010,0.001604,0.007175,...,NaN,NaN,NaN,0.007187,0.004234,0.005085,NaN,0.005810,0.002970,0.008530
25,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2018-03-25,0.880371,0.178101,0.034698,1.003906,0.006912,0.165527,0.005550,0.005070,...,NaN,NaN,NaN,0.002981,0.007481,0.007874,NaN,0.003284,0.003170,0.008514
38,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2018-03-12,0.880859,0.009705,0.004284,0.812500,0.006451,NaN,0.003796,0.007195,...,NaN,NaN,NaN,0.007381,0.006622,0.000965,NaN,0.002201,0.000834,0.003445
51,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,2018-03-29,0.621582,0.001082,0.012566,1.005859,0.007828,0.287842,0.004532,0.009941,...,NaN,NaN,NaN,0.002705,0.006184,0.001899,NaN,0.008186,0.005558,0.002983
64,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2018-03-30,0.872070,0.005573,0.007679,0.815918,0.001247,NaN,0.000231,0.005527,...,NaN,NaN,NaN,0.002974,0.004162,0.005764,NaN,0.008156,0.006943,0.000905


In [49]:
train_data_label = pd.read_csv("train_labels.csv")
# train_data_label.head()

In [50]:
train_data_final = pd.merge(train_data_4,train_data_label,how="left",on="customer_ID")

In [51]:
encoded_x = None

cat_list = train_data_final[['D_63', 'D_64', 'D_66', 'D_68', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126']]

# print(cat_list.head())

for i in range(0, cat_list.shape[1]):

    label_encoder = LabelEncoder()
    feature = label_encoder.fit_transform(cat_list.iloc[:,i])
    feature = feature.reshape(cat_list.shape[0], 1)
    onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
    feature = onehot_encoder.fit_transform(feature)
    if encoded_x is None:
        encoded_x = feature
    else:
        encoded_x = numpy.concatenate((encoded_x, feature), axis=1)
    
print("X shape:", encoded_x.shape)
print(encoded_x)

X shape: (458913, 50)
[[0. 0. 1. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 1.]
 ...
 [0. 1. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 1.]]


In [54]:
X_train = train_data_final.drop(columns=['customer_ID', 'S_2', "target",'D_63', 'D_64', 'D_66', 'D_68', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126'])
X_train_final = numpy.concatenate((X_train, encoded_x), axis=1)

y_train = train_data_final[["target"]]

model = XGBClassifier()
model.fit(X_train_final, y_train)

/opt/conda/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[22:12:40] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [55]:
test_data = pd.read_feather("test_data.ftr")

In [58]:
test_data_1 = test_data.sort_values(['customer_ID', 'S_2'], ascending = [True, False])
test_data_2 = test_data_1.drop_duplicates(subset=["customer_ID"], keep="first")

encoded_x_test = None

cat_list_test = test_data_2[['D_63', 'D_64', 'D_66', 'D_68', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126']]

# print(cat_list.head())

for i in range(0, cat_list_test.shape[1]):

    label_encoder = LabelEncoder()
    feature_test = label_encoder.fit_transform(cat_list_test.iloc[:,i])
    feature_test = feature_test.reshape(cat_list_test.shape[0], 1)
    onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
    feature_test = onehot_encoder.fit_transform(feature_test)
    if encoded_x_test is None:
        encoded_x_test = feature_test
    else:
        encoded_x_test = numpy.concatenate((encoded_x_test, feature_test), axis=1)
    
print("X shape:", encoded_x_test.shape)
print(encoded_x_test)



X shape: (924621, 50)
[[0. 0. 1. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 1. 0.]
 ...
 [0. 0. 1. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 1.]]


In [61]:
X_test = test_data_2.drop(columns=['customer_ID', 'S_2','D_63', 'D_64', 'D_66', 'D_68', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126'])
X_test_final = numpy.concatenate((X_test, encoded_x_test), axis=1)


test_out_1 = model.predict_proba(X_test_final)
test_out_1

array([[0.9456727 , 0.05432728],
       [0.9986377 , 0.00136235],
       [0.970567  , 0.02943303],
       ...,
       [0.41566467, 0.5843353 ],
       [0.8667486 , 0.1332514 ],
       [0.92571425, 0.07428574]], dtype=float32)

In [62]:
test_out_2 = pd.DataFrame(test_out_1,columns=["1_minus_pred","prediction"])
test_out_2.head()

,1_minus_pred,prediction
0,0.945673,0.054327
1,0.998638,0.001362
2,0.970567,0.029433
3,0.729215,0.270785
4,0.063967,0.936033


In [21]:
X_test_2 = test_data_3[["customer_ID"]]

cus = X_test_2["customer_ID"].reset_index()
pred = test_out_2["prediction"]

test_out_3 = pd.concat([cus["customer_ID"],pred],ignore_index=True,axis=1)
test_out_3.rename(columns = {0:'customer_ID', 1:'prediction'}, inplace = True)

test_out_3.shape

(924621, 2)

In [22]:
test_out_3.to_csv("Submission_XGboost_LatestDate_Every_Var.csv", index=False)

In [23]:
!kaggle competitions submit -c amex-default-prediction -f Submission_XGboost_LatestDate_Every_Var.csv -m "Message"

100%|██████████████████████████████████████| 67.6M/67.6M [00:04<00:00, 15.1MB/s]
Successfully submitted to American Express - Default Prediction